In [ ]:
import cv2
import numpy as np
from pylablib.devices import Thorlabs
import time
from ximea import xiapi  # Import XIMEA camera library

steps_per_mm = 34555  # in steps per mm

try:
    # Connect to the motorised stage
    stage = Thorlabs.KinesisMotor("83860050")
    print("Device connected:", stage.is_opened())

    # Initialize XIMEA camera
    cam = xiapi.Camera()  # Create camera instance
    cam.open_device()  # Open camera device
    print("XIMEA Camera connected.")

    # Set camera parameters
    cam.set_imgdataformat('XI_MONO8')  # Set image format to 8-bit grayscale
    cam.set_exposure(1000)  # Set exposure time to 1000 microseconds
    print("Exposure was set to %i us" % cam.get_exposure())

    # Create image object
    img = xiapi.Image()

    # Start image acquisition
    cam.start_acquisition()
    print("Image acquisition started.")

    def capture_image():
        """Capture an image from the XIMEA camera"""
        # Get image from the camera
        cam.get_image(img)
        
        # Get raw image data
        image_data = img.get_image_data_numpy()  # Use get_image_data_numpy() to get a NumPy array
        
        # Print the shape of the image data to check if it matches the expected size
        print(f"Image shape: {image_data.shape}")

        # If the image is grayscale, return it directly
        if len(image_data.shape) == 2:
            return image_data
        else:
            print("Unexpected image format. Expected grayscale image.")
            return None

    def calculate_spot_size(image):
        """Calculate the spot size (in pixels)"""
        # Apply binary thresholding
        _, binary = cv2.threshold(image, 50, 255, cv2.THRESH_BINARY)
       
        # Find contours
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
       
        if contours:
            # Find the largest contour
            largest_contour = max(contours, key=cv2.contourArea)
           
            # Calculate the bounding rectangle of the contour
            x, y, w, h = cv2.boundingRect(largest_contour)
           
            # Return the spot size (width or height)
            return max(w, h)
        return 0

    def check_gradient(position, step_size=1):
        """
        Check the gradient of the spot size change
        :param position: Current position (in steps)
        :param step_size: Step size (in mm)
        :return: Gradient of the spot size change (negative value means decreasing, positive means increasing)
        """
        # Move to the current position
        stage.move_to(position)
        time.sleep(0.5)
        image1 = capture_image()
        spot_size1 = calculate_spot_size(image1) if image1 is not None else 0

        # Move to the next position
        next_position = position + step_size * steps_per_mm
        stage.move_to(next_position)
        time.sleep(0.5)
        image2 = capture_image()
        spot_size2 = calculate_spot_size(image2) if image2 is not None else 0

        # Calculate the gradient
        gradient = (spot_size2 - spot_size1) / step_size
        return gradient

    def binary_search_focus(target_spot_size, tolerance=10, search_range=20, max_iterations=20):
        """
        Binary search algorithm for auto-focusing
        :param target_spot_size: Target spot size (in pixels)
        :param tolerance: Allowed error in spot size (in pixels)
        :param search_range: Search range (in mm)
        :param max_iterations: Maximum number of iterations
        """
        initial_position = stage.get_position()  # Current position

        # Check if the initial spot size is within a valid range
        stage.move_to(initial_position)
        time.sleep(0.5)  # Wait for the stage to stabilize
        initial_image = capture_image()
        if initial_image is None:
            raise ValueError("Failed to capture initial image.")
        
        initial_spot_size = calculate_spot_size(initial_image)
        print(f"Initial spot size: {initial_spot_size}")

        if initial_spot_size < 0 or initial_spot_size >= 1280:
            raise ValueError(f"Initial spot size {initial_spot_size} is out of range (0 to 1280). Algorithm will not run.")

        left = min(initial_position + (search_range * steps_per_mm) / 2, 45 * steps_per_mm)  # Left boundary of the search range
        right = max(initial_position - (search_range * steps_per_mm) / 2, 0)  # Right boundary of the search range
        best_position = initial_position
        best_error = float('inf')

        # Check the gradient until the spot size starts to increase
        current_position = initial_position
        while current_position >= right:
            gradient = check_gradient(current_position)
            if gradient > 0:
                break  # Spot size starts to increase, stop checking
            current_position += steps_per_mm * 5  # Check with a step size of 5 mm

        # Perform binary search at the position where the spot size starts to increase
        left = current_position + (search_range * steps_per_mm) / 2  # Adjust the left boundary
        right = current_position - (search_range * steps_per_mm) / 2  # Adjust the right boundary
        for i in range(max_iterations):
            mid = (left + right) / 2  # Mid position
            mid_mm = mid / steps_per_mm
            print(f"Iteration {i + 1}: Moving to position {mid_mm:.3f} mm")
            stage.move_to(mid)  # Move to the mid position
            time.sleep(0.5)  # Wait for the stage to stabilize
            image = capture_image()
            if image is not None:
                spot_size = calculate_spot_size(image)  # Calculate the spot size (in pixels)
                print(f"Spot size: {spot_size}")
                if spot_size < 0 or spot_size >= 1280:
                    print(f"Spot size {spot_size} is out of range (0 to 1280). Skipping this position.")
                    continue  # Skip positions with out-of-range spot sizes

                error = abs(spot_size - target_spot_size)  # Calculate the error (in pixels)
                if error < best_error:
                    best_error = error
                    best_position = mid
                if error <= tolerance:
                    break  # Target accuracy reached, stop searching

                # Update the search range
                if spot_size < target_spot_size:
                    right = mid  # Spot size is too small, search to the left
                else:
                    left = mid  # Spot size is too large, search to the right

        # Move to the best focus position
        print(f"Moving to best focus position: {best_position / steps_per_mm:.3f} mm")
        stage.move_to(best_position)
        print(f"Best focus position: {best_position / steps_per_mm:.3f} mm (error: {best_error} pixels)")

    # Target spot size (needs to be determined experimentally in advance)
    target_spot_size = 1021

    # Perform auto-focusing
    binary_search_focus(target_spot_size)

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Release resources
    if 'cam' in locals():
        cam.stop_acquisition()  # Stop image acquisition
        cam.close_device()  # Close the XIMEA camera
    if 'stage' in locals():
        stage.close()  # Close the stage
    cv2.destroyAllWindows()